In [1]:
import org.apache.spark.sql.functions._


val df1 = sc.parallelize(List(
  (50, 2),
  (34, 4)
)).toDF("age", "children")

val df2 = sc.parallelize(List(
  (26, true, 60000.00),
  (32, false, 35000.00)
)).toDF("age", "education", "income")


val cols1 = df1.columns.toSet
val cols2 = df2.columns.toSet

val total = cols1 ++ cols2


Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.100:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1705334395235)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
df1: org.apache.spark.sql.DataFrame = [age: int, children: int]
df2: org.apache.spark.sql.DataFrame = [age: int, education: boolean ... 1 more field]
cols1: scala.collection.immutable.Set[String] = Set(age, children)
cols2: scala.collection.immutable.Set[String] = Set(age, education, income)
total: scala.collection.immutable.Set[String] = Set(age, children, education, income)


In [2]:
// Logic is to iterate over all combined columnd and check if each col from combined col list is there in the df 
// or not. If exists then use col as is else use null vals.

def combine_columns(df_cols: Set[String], total_cols: Set[String]) = {
    total_cols.toList.map(x => x match {
        case x if df_cols.contains(x) => col(x)
        case _ => lit(null).as(x)
    })
}


combine_columns: (df_cols: Set[String], total_cols: Set[String])List[org.apache.spark.sql.Column]


In [6]:
val result_df = df1.select(combine_columns(cols1, total): _*).union(df2.select(combine_columns(cols2, total): _*))

result_df.show(false)
result_df.explain()

+---+--------+---------+-------+
|age|children|education|income |
+---+--------+---------+-------+
|50 |2       |null     |null   |
|34 |4       |null     |null   |
|26 |null    |true     |60000.0|
|32 |null    |false    |35000.0|
+---+--------+---------+-------+

== Physical Plan ==
Union
:- *(1) Project [_1#3 AS age#8, _2#4 AS children#9, null AS education#157, null AS income#158]
:  +- *(1) SerializeFromObject [knownnotnull(assertnotnull(input[0, scala.Tuple2, true]))._1 AS _1#3, knownnotnull(assertnotnull(input[0, scala.Tuple2, true]))._2 AS _2#4]
:     +- Scan[obj#2]
+- *(2) Project [_1#16 AS age#23, null AS children#159, _2#17 AS education#24, _3#18 AS income#25]
   +- *(2) SerializeFromObject [knownnotnull(assertnotnull(input[0, scala.Tuple3, true]))._1 AS _1#16, knownnotnull(assertnotnull(input[0, scala.Tuple3, true]))._2 AS _2#17, knownnotnull(assertnotnull(input[0, scala.Tuple3, true]))._3 AS _3#18]
      +- Scan[obj#15]




result_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [age: int, children: int ... 2 more fields]
